In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data/
# %ls
# %mkdir gpt2_casual

/content/drive/MyDrive/data


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from transformers import AutoTokenizer, GPT2DoubleHeadsModel
import torch
import json


# Load the JSON file
with open('finalWords.json') as f:
    words = json.load(f)

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained("gpt2")
from transformers import AutoModelForMaskedLM

model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
model = model.to(device)

original_vocab_size = len(tokenizer)
new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
num_new_words = len(new_word_list)

num_added = tokenizer.add_tokens(new_word_list)

model.resize_token_embeddings(len(tokenizer))

# new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
# num_new_words = len(new_word_list)
# num_added = tokenizer.add_tokens(new_word_list)
# model.resize_token_embeddings(len(tokenizer))

# #Compute the distribution from which we’ll sample:
# params = model.state_dict()
# embeddings = params['transformer.wte.weight']
# pre_expansion_embeddings = embeddings[:-num_new_words,:]
# mu = torch.mean(pre_expansion_embeddings, dim=0)
# n = pre_expansion_embeddings.size()[0]
# sigma = ((pre_expansion_embeddings - mu).T @ (pre_expansion_embeddings - mu)) / n
# dist = torch.distributions.multivariate_normal.MultivariateNormal(
#         mu, covariance_matrix=1e-5*sigma)

# #load in our new embeddings into the model:
# new_embeddings = torch.stack(tuple((dist.sample() for _ in range(num_new_words))), dim=0)
# embeddings[-num_new_words:,:] = new_embeddings
# params['transformer.wte.weight'][-num_new_words:,:] = new_embeddings
# model.load_state_dict(params)


Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(100258, 768)

In [ ]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
text = load_dataset("text",data_files={"train": "finetune_train.txt", "test": "finetune_val.txt"})
# print(type(text["train"]))
# text["train"] = text["train"].train_test_split(test_size=0.02)["test"]
# text["test"] = text["test"].train_test_split(test_size=0.02)["test"]
# text["test"] = text["test"][:
# print(text['train'])
# text = text['train'].train_test_split(test_size=0.2)
print(len(text["train"]), len(text["test"]))
# print(text["train"][0:10])
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# tokenizer.pad_token = tokenizer.eos_token
def preprocess_function(examples):
    return tokenizer([x for x in examples["text"]], truncation=True)
tokenized_txt = text.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=text["train"].column_names
)
# print(tokenized_txt["train "][0:100])
block_size = 128

# print(tokenizer("My wife had a PMS Strike last night. PMS strikes are ruining my marriage!"))
def group_texts(examples):
    # print(examples)
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_dataset = tokenized_txt.map(group_texts, batched=True, num_proc=4)


  0%|          | 0/2 [00:00<?, ?it/s]

75001 12500


In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
tokenizer.mask_token = u"\u2047"

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
import evaluate
# model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")
# model = GPT2LMHeadModel.from_pretrained('gpt2')
# model = model.to(device)
# metric = evaluate.load("perplexity")
# def compute_metrics(eval_pred):
#     print(eval_pred)
#     predictions, labels = eval_pred
#     predictions = predictions[:, 0]
#     return metric.compute(predictions=predictions, references=labels, model_id='gpt2')

training_args = TrainingArguments(
    output_dir="gpt2_finetune_random",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    save_steps = 10000,
    num_train_epochs=8,
    weight_decay=0.1,
    # save_total_limit=1,
    # metric_for_best_model='eval_loss',
    save_strategy="epoch",
    # logging_strategy="epoch",
    eval_accumulation_steps = 1
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()
# print(trainer.evaluate(lm_dataset["test"]))
trainer.save_model("gpt2_finetune_random")
tokenizer.save_pretrained("gpt2_finetune_random/tokenizer.json")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,4.357400,No log
2,4.069100,No log
3,3.908600,No log
4,3.820100,No log
5,3.740500,No log
6,3.677000,No log
7,3.628500,No log
8,3.617900,No log


('gpt2_finetune_random/tokenizer.json/tokenizer_config.json',
 'gpt2_finetune_random/tokenizer.json/special_tokens_map.json',
 'gpt2_finetune_random/tokenizer.json/vocab.json',
 'gpt2_finetune_random/tokenizer.json/merges.txt',
 'gpt2_finetune_random/tokenizer.json/added_tokens.json',
 'gpt2_finetune_random/tokenizer.json/tokenizer.json')

In [ ]:
#https://github.com/ayoolaolafenwa/TrainNLP

from accelerate import Accelerator
from torch.optim import AdamW
from torch import optim
from transformers import get_scheduler
from torch.utils.data import DataLoader


batch_size = 16
train_dataloader = DataLoader(lm_dataset["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator,)

# load the test dataset for evaluation
eval_dataloader = DataLoader(lm_dataset["test"], batch_size=batch_size, collate_fn=data_collator)
eval_dataset = lm_dataset["test"]
# initialize pretrained bert model
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

# set the optimizer
optimizer = AdamW(model.parameters(), lr=3e-4)

# initialize accelerator for training
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

# set the number of epochs which is set to 30
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

# define the learning rate scheduler for training
lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=num_training_steps)



ModuleNotFoundError: ignored

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 11.1 MB/s eta 0:00:00


In [ ]:

# model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
optimizer = AdamW(model.parameters(), lr=3e-4)

checkpoint = torch.load("gpt2_casual/2")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 14.5 MB/s eta 0:00:00


In [ ]:
import torch
import math
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# directory to save the models
output_dir = "gpt2_casual/"
val_loss = 1000000
num_train_epochs = 10
for epoch in range(num_train_epochs):
    # torch.save({'epoch': epoch,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict()}, 
	  #   output_dir + str(epoch))
  

    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]

    # perplexity metric used for mask language model training
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")
    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save model
    accelerator.wait_for_everyone()
    torch.save({'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': perplexity}, 
	    output_dir + str(epoch))

    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
    # if val_loss < perplexity:
    #   model.save_pretrained(output_dir)
    #   tokenizer.save_pretrained(output_dir+ "tokenizer.json")
    #   print("done training")
    #   break
    val_loss = perplexity


  0%|          | 0/15430 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


>>> Epoch 0: Perplexity: 78.94696685527958
>>> Epoch 1: Perplexity: 69.26831026735034
>>> Epoch 2: Perplexity: 68.82121601422732
>>> Epoch 3: Perplexity: 72.56297623521296
>>> Epoch 4: Perplexity: 82.03366932844374
>>> Epoch 5: Perplexity: 90.25296099530715
>>> Epoch 6: Perplexity: 106.56402674146872
>>> Epoch 7: Perplexity: 123.73160668308012
>>> Epoch 8: Perplexity: 139.3609749092947
>>> Epoch 9: Perplexity: 156.26365335801756


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-1ee05687b3bc>", line 50, in <module>
    torch.save({'epoch': epoch,
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 422, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 309, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 287, in __init__
    super(_open_zipfile_writer_file, self).__init__(torch._C.PyTorchFileWriter(str(name)))
RuntimeError: Parent directory gpt2_casual does not exist.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.p

RuntimeError: ignored

In [ ]:
# model.save_pretrained("gpt2_casual")
tokenizer.save_pretrained(output_dir+ "tokenizer.json")

NameError: ignored

In [ ]:
trainer.save_model("gpt2_mlm_model")
tokenizer.save_pretrained("gpt2_mlm_model/tokenizer.json")


('gpt2_mlm_model/tokenizer.json/tokenizer_config.json',
 'gpt2_mlm_model/tokenizer.json/special_tokens_map.json',
 'gpt2_mlm_model/tokenizer.json/vocab.json',
 'gpt2_mlm_model/tokenizer.json/merges.txt',
 'gpt2_mlm_model/tokenizer.json/added_tokens.json',
 'gpt2_mlm_model/tokenizer.json/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

model = GPT2DoubleHeadsModel.from_pretrained('gpt2_mlm_average')
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained('gpt2_mlm_average')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import json

num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
# print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

num_choices_arr = [2 , 5, 10, 25]

for num_choices in num_choices_arr:

  with open(f"quiz_questions_{num_choices}.txt") as f:
    quiz_questions = json.load(f)
  print("NUM_CHOICES:", num_choices)

 

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  counts = [0.0 for i in range(0, num_choices)]
  total = 0.0
  # print("question", len(questions))
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        if i % 1000 == 0 and i != 0:
          print(f"On iteration {i}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
                break
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            counts[mc_logits.argmax().item()] += 1
            total += 1
        torch.cuda.empty_cache() 
  print(f"FINAL for num_choices {num_choices}: accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])

  # print(counts)
  # print(total)

NUM_CHOICES: 2
On iteration 1000, accuracy is 0.47647647647647645 equal would be 0.5 count is 999.0 [0.47647647647647645, 0.5235235235235235]
On iteration 2000, accuracy is 0.47183098591549294 equal would be 0.5 count is 1988.0 [0.47183098591549294, 0.528169014084507]
On iteration 3000, accuracy is 0.4837412001340932 equal would be 0.5 count is 2983.0 [0.4837412001340932, 0.5162587998659068]
On iteration 4000, accuracy is 0.4847951746670018 equal would be 0.5 count is 3979.0 [0.4847951746670018, 0.5152048253329983]
On iteration 5000, accuracy is 0.4892505525416918 equal would be 0.5 count is 4977.0 [0.4892505525416918, 0.5107494474583082]
On iteration 6000, accuracy is 0.4884538152610442 equal would be 0.5 count is 5976.0 [0.4884538152610442, 0.5115461847389559]
On iteration 7000, accuracy is 0.490752688172043 equal would be 0.5 count is 6975.0 [0.490752688172043, 0.509247311827957]
On iteration 8000, accuracy is 0.4890227073140133 equal would be 0.5 count is 7971.0 [0.4890227073140133

In [ ]:
import json

num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
# print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

num_choices_arr = [5]
model_ans = []
for num_choices in num_choices_arr:

  with open(f"quiz_questions_{num_choices}.txt") as f:
    quiz_questions = json.load(f)
  print("NUM_CHOICES:", num_choices)

 

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  counts = [0.0 for i in range(0, num_choices)]
  total = 0.0
  # print("question", len(questions))
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        if i % 1000 == 0 and i != 0:
          print(f"On iteration {i}, model_ans length is {len(model_ans)}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
                break
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            counts[mc_logits.argmax().item()] += 1
            total += 1
            model_ans.append(mc_logits.argmax().item())
        else:
          model_ans.append(-1)
        torch.cuda.empty_cache() 
  print(f"FINAL for num_choices {num_choices}: accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])

  # print(counts)
  # print(total)

NUM_CHOICES: 5
On iteration 1000, model_ans length is 1000, accuracy is 0.1995987963891675 equal would be 0.2 count is 997.0 [0.1995987963891675, 0.20661985957873621, 0.20361083249749248, 0.20260782347041123, 0.18756268806419257]
On iteration 2000, model_ans length is 2000, accuracy is 0.20211693548387097 equal would be 0.2 count is 1984.0 [0.20211693548387097, 0.20967741935483872, 0.19606854838709678, 0.20715725806451613, 0.1849798387096774]
On iteration 3000, model_ans length is 3000, accuracy is 0.19879113498992612 equal would be 0.2 count is 2978.0 [0.19879113498992612, 0.20953660174613833, 0.19677635997313633, 0.20852921423774345, 0.18636668905305573]
On iteration 4000, model_ans length is 4000, accuracy is 0.2035264483627204 equal would be 0.2 count is 3970.0 [0.2035264483627204, 0.20554156171284635, 0.19748110831234256, 0.2047858942065491, 0.18866498740554155]
On iteration 5000, model_ans length is 5000, accuracy is 0.20395002015316405 equal would be 0.2 count is 4962.0 [0.20395

In [ ]:
f = open("compare_5/pretrain_average.txt", "w+")
for i in model_ans:
  f.write(str(i)+"\n")
f.close()